In [1]:
# Análisis
import numpy as np
import pandas as pd
import seaborn as sns
import joblib

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score

# Pipelines
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector

# Herramientas de preprocesamiento
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


In [2]:
df= sns.load_dataset('diamonds')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [ ]:
df[df['cut'] == 'Ideal'].sort_values('price', ascending=False)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
11,0.23,Ideal,J,VS1,62.8,56.0,340,3.93,3.90,2.46
13,0.31,Ideal,J,SI2,62.2,54.0,344,4.35,4.37,2.71
16,0.30,Ideal,I,SI2,62.0,54.0,348,4.31,4.34,2.68
28262,0.25,Ideal,H,SI1,62.8,54.0,357,4.05,4.07,2.55
...,...,...,...,...,...,...,...,...,...,...
27735,1.60,Ideal,F,VS1,62.0,56.0,18780,7.47,7.52,4.65
27738,2.05,Ideal,G,SI1,61.9,57.0,18787,8.10,8.16,5.03
27741,2.15,Ideal,G,SI2,62.6,54.0,18791,8.29,8.35,5.21
27746,2.07,Ideal,G,SI2,62.5,55.0,18804,8.20,8.13,5.11


In [3]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [4]:
df['clarity'].unique()

['SI2', 'SI1', 'VS1', 'VS2', 'VVS2', 'VVS1', 'I1', 'IF']
Categories (8, object): ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']

* En las columnas x, y, z, considero los valores 0 como nulos y los reemplazo por nan para luego imputarlos.

In [5]:
df= df.replace((0), np.nan)
df.isnull().sum()

carat       0
cut         0
color       0
clarity     0
depth       0
table       0
price       0
x           8
y           7
z          20
dtype: int64

In [6]:
# Borro filas duplicadas
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())

146
0


In [7]:
X = df.drop(['price'], axis=1)
y = df['price']

categorical_cols = ['cut', 'color', 'clarity']
numerical_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']


In [8]:
numerical_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler()
)

categorical_pipeline = make_pipeline(OneHotEncoder())

column_transformer = make_column_transformer(
    (numerical_pipeline, numerical_cols),
    (categorical_pipeline, categorical_cols)
)
pipeline = make_pipeline(column_transformer, GradientBoostingRegressor(max_depth=10))

pipeline.fit(X, y)
print('R2 en train', pipeline.score(X, y))

R2 en train 0.9947864698815451


In [9]:
joblib.dump(pipeline, 'pipeline_regresion.joblib')

['pipeline_regresion.joblib']